In [1]:
import os
os.environ['DGLBACKEND'] = 'pytorch'
import numpy as np
import pandas as pd
import h5py
import torch
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm
import dgl
from compute_edges import compute_one_edge_feature

In [2]:
h5_file = h5py.File('../data/train/jetImage_0_100p_0_10000.h5', 'r')
mask = np.sum(np.abs(h5_file["jetConstituentList"]), axis=2)==0
print(mask.shape)
h5_file.keys()

(10000, 100)


<KeysViewHDF5 ['jetConstituentList', 'jetFeatureNames', 'jetImage', 'jetImageECAL', 'jetImageHCAL', 'jets', 'particleFeatureNames']>

In [33]:
h5_file['jetFeatureNames'][:]

array([b'j_ptfrac', b'j_pt', b'j_eta', b'j_mass', b'j_tau1_b1',
       b'j_tau2_b1', b'j_tau3_b1', b'j_tau1_b2', b'j_tau2_b2',
       b'j_tau3_b2', b'j_tau32_b1', b'j_tau32_b2', b'j_zlogz', b'j_c1_b0',
       b'j_c1_b1', b'j_c1_b2', b'j_c2_b1', b'j_c2_b2', b'j_d2_b1',
       b'j_d2_b2', b'j_d2_a1_b1', b'j_d2_a1_b2', b'j_m2_b1', b'j_m2_b2',
       b'j_n2_b1', b'j_n2_b2', b'j_tau1_b1_mmdt', b'j_tau2_b1_mmdt',
       b'j_tau3_b1_mmdt', b'j_tau1_b2_mmdt', b'j_tau2_b2_mmdt',
       b'j_tau3_b2_mmdt', b'j_tau32_b1_mmdt', b'j_tau32_b2_mmdt',
       b'j_c1_b0_mmdt', b'j_c1_b1_mmdt', b'j_c1_b2_mmdt', b'j_c2_b1_mmdt',
       b'j_c2_b2_mmdt', b'j_d2_b1_mmdt', b'j_d2_b2_mmdt',
       b'j_d2_a1_b1_mmdt', b'j_d2_a1_b2_mmdt', b'j_m2_b1_mmdt',
       b'j_m2_b2_mmdt', b'j_n2_b1_mmdt', b'j_n2_b2_mmdt', b'j_mass_trim',
       b'j_mass_mmdt', b'j_mass_prun', b'j_mass_sdb2', b'j_mass_sdm1',
       b'j_multiplicity', b'j_g', b'j_q', b'j_w', b'j_z', b'j_t',
       b'j_undef'], dtype=object)

In [3]:

class DataReader:
    
    def __init__(self, path):
        
        self.path = path
        self.data = None
        
    def get_filenames(self):
        
        fnames = os.listdir(self.path)
        fnames = [filename for filename in fnames if filename.endswith(".h5")]

        return fnames
    
    
    def read_single_file(self, fname):
        
        file_path = os.path.join(self.path, fname)
        h5_file = h5py.File(file_path, "r")
        
        # turn "jets" into a dataframe with column names given by "jetFeaturesNames"

        jet_feature_names = h5_file["jetFeatureNames"][:]

        # remove the b' from the beginning and the ' from the end of each string
        jet_feature_names = [name.decode("utf-8") for name in jet_feature_names]

        df = pd.DataFrame(h5_file["jets"][:], columns=jet_feature_names)

        # keep features
        keep = ["j_g", "j_q", "j_w", "j_z", "j_t", "j_undef"]

        # rename features to "isGluon", "isQuark", "isW", "isZ", "isTop", "isUndefined"
        df = df[keep].rename(columns={"j_g": "isGluon", "j_q": "isQuark", "j_w": "isW", "j_z": "isZ", "j_t": "isTop", "j_undef": "isUndefined"}).astype(int)


        # LABELS
        labels = df.values

        mask = np.sum(np.abs(h5_file["jetConstituentList"]), axis=2)==0
        
        # FEATURES
        e      = np.ma.masked_array(h5_file["jetConstituentList"][:, :, 3],  mask) # E
        e_rel  = np.ma.masked_array(h5_file["jetConstituentList"][:, :, 4],  mask) # E
        pt     = np.ma.masked_array(h5_file["jetConstituentList"][:, :, 5],  mask) # pT
        pt_rel = np.ma.masked_array(h5_file["jetConstituentList"][:, :, 6],  mask) # pT particle / jet
        dEta   = np.ma.masked_array(h5_file["jetConstituentList"][:, :, 8],  mask) # dEta
        dPhi   = np.ma.masked_array(h5_file["jetConstituentList"][:, :, 11], mask) # dPhi
        dR     = np.ma.masked_array(h5_file["jetConstituentList"][:, :, 13], mask) # dR
        

        return {"labels": labels,"e": e, "e_rel": e_rel, "pt": pt, "pt_rel": pt_rel, "dEta": dEta, "dPhi": dPhi, "dR": dR}


    
    def read_files(self, n_files=None):
        
        fnames = self.get_filenames()
        
        if n_files is not None:
            fnames = fnames[:n_files]
            
        for i, fname in enumerate(fnames):
            
            print("Reading file", fname)
            
            data = self.read_single_file(fname)
            
            if i == 0:
                labels = data["labels"]
                e      = data["e"]
                e_rel  = data["e_rel"]
                pt     = data["pt"]
                pt_rel = data["pt_rel"]
                dEta   = data["dEta"]
                dPhi   = data["dPhi"]
                dR     = data["dR"]
                
            else:
                labels = np.ma.concatenate((labels, data["labels"]))
                e      = np.ma.concatenate((e, data["e"]))
                e_rel  = np.ma.concatenate((e_rel, data["e_rel"]))
                pt     = np.ma.concatenate((pt, data["pt"]))
                pt_rel = np.ma.concatenate((pt_rel, data["pt_rel"]))
                dEta   = np.ma.concatenate((dEta, data["dEta"]))
                dPhi   = np.ma.concatenate((dPhi, data["dPhi"]))
                dR     = np.ma.concatenate((dR, data["dR"]))
    
        self.data = {"labels": labels,"e": e, "e_rel": e_rel, "pt": pt, "pt_rel": pt_rel, "dEta": dEta, "dPhi": dPhi, "dR": dR}        
    
    
    def get_labels(self):
        if self.data:
            return self.data["labels"]
        else:
            raise RuntimeError('No data read, please call .read_files before!')
    
    def get_features(self):
        if self.data:
            return np.ma.stack(
                (
                    self.data["dEta"],
                    self.data["dPhi"],
                    self.data["e"],
                    self.data["e_rel"],
                    self.data["pt"],
                    self.data["pt_rel"],
                    self.data["dR"],
                ),
                axis=2,
            )
        else:
            raise RuntimeError('No data read, please call .read_files before!')

In [8]:
def node_feature_one_jet(jet):
    mask = np.tile(np.array([False, False, True, True, True, True, False]), (jet.shape[0], 1))
    result = jet.copy()
    np.log(jet, where=mask, out=result)
    return result

In [36]:
data_reader = DataReader("../data/train/")
data = data_reader.read_single_file("jetImage_5_100p_10000_20000.h5")
jets = np.ma.stack(
                (
                    data["dEta"],
                    data["dPhi"],
                    data["e"],
                    data["e_rel"],
                    data["pt"],
                    data["pt_rel"],
                    data["dR"],
                ),
                axis=2,
            )

jets_labels = torch.tensor(data['labels'])
jets

masked_array(
  data=[[[0.016838908195495605, 0.023335546255111694,
          167.12623596191406, ..., 151.68331909179688,
          0.12082843482494354, 0.028776662424206734],
         [0.016711080446839333, 0.02542714774608612, 116.93895721435547,
          ..., 106.13909912109375, 0.08454865962266922,
          0.030426958575844765],
         [0.012246585451066494, 0.026666536927223206, 88.78760528564453,
          ..., 80.73810577392578, 0.06431464850902557,
          0.029344214126467705],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[-0.01816887967288494, 0.11409587413072586, 318.28619384765625,
          ..., 254.62315368652344, 0.2786259651184082,
          0.11553344130516052],
         [0.07251614332199097, -0.158086895942688, 118.0810775756836,
          ..., 99.4735107421875, 0.10885067284107208,
          0.17392544448375702],
         [-0.00998386275023222, 0.10954894870519

In [6]:
jets_labels.shape

(10000, 6)

In [21]:
corrupted_jet = jets[1729]
corrupted_jet.compressed().reshape((-1, corrupted_jet.shape[-1]))[5]

array([-1.94004729e-01,  2.86700100e-01,  7.38627136e-01,  6.50253380e-04,
        5.89623928e-01,  5.78226463e-04,  3.46171588e-01])

In [5]:
jets = data_reader.get_features()

In [57]:
graphs = []
for jet, label in zip(tqdm(jets[:1000]), jets_labels):

    jet = jet.compressed().reshape((-1, jet.shape[-1]))
    edges_features = []
    nodes_features = []
    sources = []
    destinations = []
    for i in range(jet.shape[0]):
        for j in range(jet.shape[0]):

            edge_features = np.ones(3) if i == j else compute_one_edge_feature(jet, i, j)
           
            edges_features.append(edge_features)
            sources.append(i)
            destinations.append(j)
        nodes_features.append(node_feature_one_jet(jet)[i])

    g = dgl.graph((torch.tensor(sources), torch.tensor(destinations)))
    g.edata['d'] = torch.tensor(np.stack(edges_features), dtype=torch.float32)
    g.ndata['f'] = torch.tensor(np.stack(nodes_features), dtype=torch.float32)
    g.ndata['labels'] = label.unsqueeze(0).expand((jet.shape[0], 6))
    graphs.append(g)



  0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
graphs[0].ndata['labels']=jets_labels[0]

DGLError: Expect number of features to match number of nodes (len(u)). Got 6 and 65 instead.

In [45]:
jets_labels.shape

torch.Size([10000, 6])

In [51]:
dgl.save_graphs('graphs.dgl', graphs, labels={'tag':jets_labels})

In [54]:
loaded_graphs = dgl.load_graphs('../data/graphdataset/graphs0-20000.dgl')

In [56]:
loaded_graphs[1]['tag'].shape

torch.Size([20000, 6])

In [50]:
labels[0]

array(['Gluon', tensor([1, 0, 0,  ..., 0, 0, 0])], dtype=object)

## FUCK

In [46]:
jet = jet.compressed().reshape((-1, jet.shape[-1]))

In [49]:
node_feature_one_jet(jet)[0]

array([-0.01986081, -0.08190626,  5.24482898, -1.80925894,  5.21972721,
       -1.801706  ,  0.08427981])

In [50]:
jet[0]

array([-1.98608059e-02, -8.19062591e-02,  1.89583389e+02,  1.63775459e-01,
        1.84883743e+02,  1.65017128e-01,  8.42798129e-02])

In [37]:
np.log(jet[0], where=[False, False, True, True, True, True, False])

array([ 0.01314407,  0.04510712,  6.0447516 , -1.11842318,  5.82647504,
       -1.1238389 ,  0.04698318])

In [39]:
np.tile(np.array([False, False, True, True, True, True, False]), (jet.shape[0], 1))

(40, 7)

## Optimize computing edge features

In [67]:
def compute_dPhi_ij(dPhi):
    """Compute the difference in azimuthal angle between two particles."""
    expanded_dPhi = np.tile(dPhi, (dPhi.shape[0], 1))
    return expanded_dPhi - expanded_dPhi.T

def compute_dEta_ij(dEta):
    """Compute the difference in pseudorapidity between two particles."""
    expanded_dEta = np.tile(dEta, (dEta.shape[0], 1))
    return expanded_dEta - expanded_dEta.T

def compute_R_ij(dEta_ij, dPhi_ij):
    """Compute the distance between two particles in the transverse plane."""
    return np.sqrt(dEta_ij**2 + dPhi_ij**2)

def compute_m_ij(pt, dEta_ij, dPhi_ij):
    """Compute the invariant mass of two particles."""
    # invariant mass of two massive particles as a function of the two transverse momenta and the angles between them
    expanded_pt = np.tile(pt, (pt.shape[0], 1))
    pt_ij = expanded_pt * expanded_pt.T
    return np.sqrt(2. * pt_ij * np.cosh(dEta_ij) - 2. * pt_ij * np.cos(dPhi_ij)) # RELATIVISTIC APPROX

def node_distance(pt, r, r_ij, alpha):
    """Compute the distance between two nodes in the graph."""
    expanded_pt = np.tile(pt, (pt.shape[0], 1))
    return np.minimum(expanded_pt**(2*alpha), expanded_pt.T**(2*alpha)) * r_ij**2/r**2

def compute_1jet_edge_features(jet):
    """Compute the edge feature for one edge."""
    
    dEta_ij = compute_dEta_ij(jet[:, 0])
    dPhi_ij = compute_dPhi_ij(jet[:, 1])
    dR_ij   = compute_R_ij(dEta_ij, dPhi_ij)
    m_ij    = compute_m_ij(jet[:, 4], dEta_ij, dPhi_ij)

    # compute the edge feature
    e_0 = node_distance(pt=jet[:, 4], r=np.max(jet[:, 6]), r_ij=dR_ij, alpha=0)
    e_1 = node_distance(pt=jet[:, 4], r=np.max(jet[:, 6]), r_ij=dR_ij, alpha=1)
    e_2 = m_ij

    # fill with ones on the diagonal
    np.fill_diagonal(e_0, 1.)
    np.fill_diagonal(e_1, np.e)
    np.fill_diagonal(e_2, np.e)

    if np.any(e_2 == 0.):
        print(f'got zero m_ij')
        e_2[e_2==0] = 1e-6

    if np.any(e_1 == 0.):
        print(f'got zero node_distance')
        e_1[e_1==0] = 1e-6

    e_1 = np.log(e_1)
    e_2 = np.log(e_2)

    # flatten
    e_0 = e_0.flatten()
    e_1 = e_1.flatten()
    e_2 = e_2.flatten()

    return np.stack([e_0, e_1, e_2], 1)
    




In [68]:
data_reader = DataReader("../data/train/")
data_reader.read_files(2)

jets = data_reader.get_features()

for jet in tqdm(jets):
    jet = jet.compressed().reshape((-1, jet.shape[-1]))

    edge_feat = compute_1jet_edge_features(jet)

Reading file jetImage_0_100p_0_10000.h5
Reading file jetImage_0_100p_10000_20000.h5


  0%|          | 0/20000 [00:00<?, ?it/s]

In [28]:
edge_feat.shape

(10000, 3)

In [54]:
torch.arange(0, 6).unsqueeze(0).expand((6, 6)).flatten()

tensor([0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5,
        0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5])

In [64]:
dPhi_exp[dPhi_exp < 0] = np.nan

NameError: name 'dPhi_exp' is not defined

In [46]:
np.argmax(jets[:, :, 6])

452358

In [52]:
jets[4523, 50:70, 6]

masked_array(data=[0.08965777605772018, 0.04055321216583252,
                   0.01901513896882534, 0.10037165135145187,
                   0.15630632638931274, 0.04033689200878143,
                   0.36087700724601746, 0.18934768438339233,
                   1.5411721467971802, --, --, --, --, --, --, --, --, --,
                   --, --],
             mask=[False, False, False, False, False, False, False, False,
                   False,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True],
       fill_value=1e+20)

In [7]:
x = np.array([[1, 2], [3, 4]])
y = np.array([[5, 6], [7, 8]])
x.flatten()

array([1, 2, 3, 4])

In [8]:
np.stack([x.flatten(), y.flatten()], 1)[0, :]

array([1, 5])

In [65]:
x[x<3] = 5

In [66]:
x

array([[5, 5],
       [3, 4]])